Load the data

In [4]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [5]:
reviewers_df = pd.read_csv('../data/users_with_locations.csv.gz',compression='gzip')

explore the data - not too much class imbalance -- XGboost will take care of this.

In [6]:
# look at class balance of the column location_us
print(reviewers_df['location_us'].value_counts())


location_us
0    333266
1    149620
Name: count, dtype: int64


using spacy lets egineer some more features

In [7]:
%pip install spacy
%pip install re 
%pip install tqdm
%pip install numpy
%pip install -U pip setuptools wheel
%pip install -U spacy
%pip install -U spacy-transformers

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import spacy
import re
import numpy as np
from tqdm import tqdm

In [9]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.0 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


0

In [10]:
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [12]:
# Load spaCy transformer model
print("Loading spaCy model...")
nlp = spacy.load("en_core_web_sm")

# US cities and states for validation/lookup
us_states = set([
    "alabama", "alaska", "arizona", "arkansas", "california", "colorado", 
    "connecticut", "delaware", "florida", "georgia", "hawaii", "idaho", 
    "illinois", "indiana", "iowa", "kansas", "kentucky", "louisiana", 
    "maine", "maryland", "massachusetts", "michigan", "minnesota", 
    "mississippi", "missouri", "montana", "nebraska", "nevada", 
    "new hampshire", "new jersey", "new mexico", "new york", "north carolina", 
    "north dakota", "ohio", "oklahoma", "oregon", "pennsylvania", 
    "rhode island", "south carolina", "south dakota", "tennessee", "texas", 
    "utah", "vermont", "virginia", "washington", "west virginia", 
    "wisconsin", "wyoming", "dc", "district of columbia"
])
# Add state abbreviations
state_abbrevs = {
    "al", "ak", "az", "ar", "ca", "co", "ct", "de", "fl", "ga", "hi", "id",
    "il", "in", "ia", "ks", "ky", "la", "me", "md", "ma", "mi", "mn", "ms",
    "mo", "mt", "ne", "nv", "nh", "nj", "nm", "ny", "nc", "nd", "oh", "ok",
    "or", "pa", "ri", "sc", "sd", "tn", "tx", "ut", "vt", "va", "wa", "wv",
    "wi", "wy", "dc"
}
us_states.update(state_abbrevs)

# Top 100 US cities by population
top_us_cities = {
    "new york", "los angeles", "chicago", "houston", "phoenix", "philadelphia",
    "san antonio", "san diego", "dallas", "san jose", "austin", "jacksonville",
    "fort worth", "columbus", "charlotte", "indianapolis", "san francisco",
    "seattle", "denver", "washington", "boston", "el paso", "nashville",
    "detroit", "oklahoma city", "portland", "las vegas", "memphis", "louisville",
    "baltimore", "milwaukee", "albuquerque", "tucson", "fresno", "sacramento",
    "kansas city", "mesa", "atlanta", "omaha", "colorado springs", "raleigh",
    "miami", "long beach", "virginia beach", "oakland", "minneapolis", "tampa",
    "tulsa", "arlington", "wichita", "cleveland", "bakersfield", "aurora", 
    "new orleans", "honolulu", "anaheim", "tampa", "pittsburgh", "cincinnati",
    "st louis", "riverside", "corpus christi", "lexington", "anchorage",
    "stockton", "toledo", "st paul", "newark", "greensboro", "buffalo", 
    "plano", "lincoln", "henderson", "fort wayne", "jersey city", "st petersburg",
    "chula vista", "orlando", "durham", "chandler", "laredo", "madison", "lubbock",
    "scottsdale", "reno", "glendale", "boise", "richmond", "baton rouge", "irvine", 
    "spokane", "tacoma", "irving", "hialeah", "fremont", "birmingham",
    "rochester", "san bernardino", "boise city"
}
state_pattern = re.compile(r"[A-Za-z\s]+,\s*([A-Z]{2})")
zip_pattern = re.compile(r"\b\d{5}(?:-\d{4})?\b")

# Ensure consistent language detection results
DetectorFactory.seed = 42

def is_english(text):
    """
    Detect if the given text is in English.
    Returns True if the text is in English, False otherwise.
    """
    try:
        return detect(text) == "en"
    except LangDetectException:
        return False

# Function to check if a location is in the US
def is_us_location(location_text):
    if not location_text:
        return False
        
    location_lower = location_text.lower()
    
    # Direct state or city match
    if any(state in location_lower.split() for state in us_states) or \
       any(city in location_lower for city in top_us_cities):
        return True
    
    # Check for state pattern (City, ST)
    state_match = state_pattern.search( location_text)
    if state_match and state_match.group(1).lower() in state_abbrevs:
        return True
    
    # Check for zip code pattern
    if zip_pattern.search(location_text):
        return True
    
    # Check for USA/America mentions
    us_terms = ["usa", "america", "united states", "u.s.", "u.s.a."]
    if any(term in location_lower for term in us_terms):
        return True
        
    return False

# Function to extract location from bio text using spaCy
def extract_locations_in_batch(texts, nlp_model):
    """
    Extract locations from a batch of texts using spaCy's pipe.
    """
    locations = []
    for doc in nlp_model.pipe(texts, batch_size=1000):  # Adjust batch_size as needed
        locs = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]
        locations.append(locs[0] if locs else "nil")
    return locations

# Function to add location columns to dataframe
def add_location_columns(df, nlp_model, batch_size=1000):
    """
    Process dataframe and add location columns using spaCy's pipe for batch processing.


    """
    #Replace NaN or non-string values in 'full_bio' with an empty string
    df['full_bio'] = df['full_bio'].fillna("").astype(str)

    # Detect language and filter for English bios -show a progress bar
    tqdm.pandas(desc="Detecting English bios")
    # Use apply with tqdm to show progress
    # Use a lambda function to apply the is_english function to each bio
    english_df = df[df['full_bio'].progress_apply(lambda x: is_english(x))]
    
    # Process in batches
    total_batches = (len(english_df) + batch_size - 1) // batch_size
    spacy_locations = []
    
    for i in tqdm(range(total_batches), desc="Processing bios"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(english_df))
        batch = english_df['full_bio'].iloc[start_idx:end_idx]
        
        # Extract locations in batch
        spacy_locations.extend(extract_locations_in_batch(batch, nlp_model))
    
    # Add extracted locations to the DataFrame
    english_df['spacy_location'] = spacy_locations
    
    # Check if the extracted location is in the US
    english_df['spacy_us_location'] = english_df['spacy_location'].apply(lambda loc: int(is_us_location(loc)))
    # Merge results back into the original DataFrame
    df = df.merge(english_df[['spacy_location', 'spacy_us_location']], how='left', left_index=True, right_index=True)
    df['spacy_location'] = df['spacy_location'].fillna("nil")
    df['spacy_us_location'] = df['spacy_us_location'].fillna(0)
    
    return df


engineered_df = add_location_columns(reviewers_df, nlp, batch_size=1000)





Loading spaCy model...


Processing bios: 100%|██████████| 213/213 [04:49<00:00,  1.36s/it]
/tmp/ipykernel_92824/2377693858.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_df['spacy_location'] = spacy_locations
/tmp/ipykernel_92824/2377693858.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_df['spacy_us_location'] = english_df['spacy_location'].apply(lambda loc: int(is_us_location(loc)))


In [ ]:
# save the processed dataframe to a new CSV file
engineered_df.to_csv('../data/engineered_users_with_locations_sm.csv.gz', index=False, compression='gzip')



In [ ]:
print(engineered_df.shape)
# couint the number of records where locattion_us is 0 but spacy_us_location is 1
print("Number of records where location_us is 0 and spacy_us_location is 1:")
print(len(engineered_df[(engineered_df['location_us'] == 0) & (engineered_df['spacy_us_location'] == 1)]))

# #show the full bio and meta columns of these records
print(engineered_df[(engineered_df['location_us'] == 1) & (engineered_df['spacy_us_location'] == 1)][['full_bio', 'metadata','spacy_location','location_us']].head())


(482886, 108)
Number of records where location_us is 0 and spacy_us_location is 1:
356
                                               full_bio  \
466   San Francisco-based Afghan-American artist and...   
1296  I run The Goodbye Look, a jazz pop-up series i...   
1432  I like standup, sad shit, and musicals and i w...   
1773   1oneam   I chat so much Yet know so little In VA   
2453  Blu Ray and Dvd Collector .. I live In New Yor...   

                                               metadata spacy_location  
466   California;1001-films.blogspot.com : https://1...  San Francisco  
1296  Seattle;petergraham.world : http://www.petergr...        Seattle  
1432  Richmond, Virginia;222sammm : https://twitter....  San Francisco  
1773                                                AVA             VA  
2453  Brooklyn, NY;youtube.com : http://youtube.com/...  New York City  


Number of records in the original dataframe:
(482886, 108)
Number of records in the processed dataframe:
(482886, 108)
